In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [4]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

# nltk.download("punkt")
import sys

sys.path.append("../")


from torchtext.datasets import WikiText2

from scipy.spatial.distance import cosine

In [7]:
data_iter = WikiText2(root="../data/", split=("train"))


def get_sent_tokens_from_iterator(data_iter):
    all_sent_tokens = []
    for line in data_iter:
        sent_tokens = []
        line_lower = line.lower()
        sentences = sent_tokenize(line_lower)
        for sent in sentences:
            sent_tokens.append(sent.split())

        if len(sent_tokens) > 0:
            all_sent_tokens.extend(sent_tokens)
    return all_sent_tokens


sent_tokens = get_sent_tokens_from_iterator(data_iter)
len(sent_tokens)

86861

In [8]:
n_tokens = 0
for x in sent_tokens:
    n_tokens += len(x)

n_tokens / 1000000

2.055104

## Training Word2Vec with Gensim

In [13]:
from gensim.models import Word2Vec

# https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

In [21]:
model = Word2Vec(
    sentences=sent_tokens,
    vector_size=300,
    window=4,
    min_count=50,
    workers=4,
    sg=0, #skipgram
    hs=0, #hierarchical softmax
    cbow_mean=1,
    alpha=0.025
    min_alpha=0.005,
    negative=0,
    ns_exponent=None,
    epochs=5,
)


44667196.0

In [44]:
model = Word2Vec(
    sentences=sent_tokens,
    vector_size=300,
    alpha=0.025,
    window=4,
    min_count=50,
    max_vocab_size=None,
    sample=0,
    seed=1,
    workers=3,
    min_alpha=0.005,
    sg=0,
    hs=1,
    negative=0,
    ns_exponent=0.0,
    cbow_mean=1,
    epochs=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    comment=None,
    max_final_vocab=None,
)

In [45]:
model.wv.most_similar("father", topn=10)

[('mother', 0.8185541033744812),
 ('daughter', 0.7746013402938843),
 ('wife', 0.7724822163581848),
 ('brother', 0.7704977989196777),
 ('son', 0.735470712184906),
 ('successor', 0.6820389032363892),
 ('husband', 0.6800978779792786),
 ('opponent', 0.6787068247795105),
 ('friend', 0.6636711955070496),
 ('fate', 0.6383050680160522)]

In [36]:
king_v = model.wv["king"]
queen_v = model.wv["queen"]
woman_v = model.wv["woman"]
man_v = model.wv["man"]

king_v = king_v / ((king_v ** 2).sum() ** (1 / 2))
queen_v = queen_v / ((queen_v ** 2).sum() ** (1 / 2))
woman_v = woman_v / ((woman_v ** 2).sum() ** (1 / 2))
man_v = man_v / ((man_v ** 2).sum() ** (1 / 2))

In [59]:
(king_v ** 2).sum()

0.9999999

In [63]:
new_v = (king_v - man_v) + woman_v

In [62]:
cosine(new_v, queen_v)

0.5770814418792725